In [98]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

df = pd.read_csv('../dataSets/training/training_20min_avg_travel_time.csv')
trajectories_train = {}
start_time  = datetime.strptime("2016-07-19 00:00:00",'%Y-%m-%d %H:%M:%S')

for idx in range(df.shape[0]):
    line = df.iloc[idx]
    route = str(line['intersection_id']+str(line['tollgate_id']))
    stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
    day = (stime - start_time).days
    tw_n = (stime.hour * 60 + stime.minute)/20
    
    if route not in trajectories_train.keys():
        trajectories_train[route] = np.zeros(72*91).reshape(91,72)
    trajectories_train[route][day][tw_n] = line['avg_travel_time']

In [99]:
def TrueFalseListCombine(TFlist1,TFlist2):
    return [l1 and l2 for l1,l2 in zip(TFlist1,TFlist2)]

In [100]:
def get_history_mean(route,twn,time):
    #time=datetime.strptime(date,'%Y-%m-%d %H:%M:%S')
    startday=(time-start_time).days
    
    history_data=[]
    while(startday>-1):
        if(startday<90):
            cur_data=trajectories_train[route][startday][twn]
            history_data.append(cur_data)
        startday-=7
    
    #for i in range(len(history_data)):
    #    print history_data[i]
    
    max_value=np.max(history_data)
    min_value=np.min(history_data)
    sum=0.0
    n=0.0
    for i in range(len(history_data)):
        if history_data[i]>min_value and history_data[i]<max_value:
            sum+=history_data[i]
            n+=1.0
            
    #mean_result = sum/n
    #sum_e = 0.0
    #for i in range(len(history_data)):
    #    if history_data[i]>min_value and history_data[i]<max_value:
    #         sum_e += abs((history_data[i]- mean_result)/history_data[i])
    #print "error:",sum_e/n
    result = 0
    if n != 0:
        result = sum/n
    return result

In [101]:
#get_history_mean("A3",54,"2016-10-19 00:00:00")

In [102]:
pstart_time=datetime.strptime("2016-10-18 00:00:00",'%Y-%m-%d %H:%M:%S')


y_tw_list = [24,25,26,27,28,29,51,52,53,54,55,56]
#y_tw_list = [18,19,20,21,22,23,45,46,47,48,49,50]
test_data = {}
for route in trajectories_train.keys():
    for twn in y_tw_list:
        for day in range(7):
            if route not in test_data.keys():
                test_data[route] = np.zeros(7*72).reshape(7,72)
            test_data[route][day][twn] = get_history_mean(route,twn,pstart_time+timedelta(days=day))

In [103]:
def out_data(test_data):
    out_data = test_data;
    ## 数据输出
    from datetime import datetime
    from datetime import timedelta
    from pandas import DataFrame


    intersection = []
    tollgate = []
    time_window = []
    avg_time = []
    routes = ['A2','A3',"B1","B3","C1","C3"]
    for route in routes:
        n_day = 7
        start_time = datetime.strptime('2016-10-18 08:00:00',"%Y-%m-%d %H:%M:%S")
        for day in range(n_day):
            starttime = start_time + timedelta(days=day)
            i = 0
            for k in range(24,30): #(51,57)（24，30）
                time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                                   + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
                intersection.append(str(route[0]))
                tollgate.append(str(route[1]))
                avg_time.append(out_data[route][day][k])
                i+=1

    for route in routes:
        n_day = 7
        start_time = datetime.strptime('2016-10-18 17:00:00',"%Y-%m-%d %H:%M:%S")
        for day in range(n_day):
            starttime = start_time + timedelta(days=day)
            i = 0
            for k in range(51,57): #(51,57)（24，30）
                time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                                   + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
                intersection.append(str(route[0]))
                tollgate.append(str(route[1]))
                avg_time.append(out_data[route][day][k])
                i+=1            

    d = {"intersection_id":intersection,"tollgate_id":tollgate,"time_window":time_window,"avg_travel_time":avg_time}
    pd = DataFrame(data=d)
    pd.to_csv('out.csv',index=False,columns=["intersection_id","tollgate_id","time_window","avg_travel_time"])

In [104]:
out_data(test_data)

In [105]:
df_test=pd.read_csv('../dataSets/testing_phase1/trajectories_20min_avg_travel_time.csv')
trajectories_test = {}
start_time  = datetime.strptime("2016-10-18 00:00:00",'%Y-%m-%d %H:%M:%S')

for idx in range(df_test.shape[0]):
    line = df_test.iloc[idx]
    route = str(line['intersection_id']+str(line['tollgate_id']))
    stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
    day = (stime - start_time).days
    tw_n = (stime.hour * 60 + stime.minute)/20
    
    if route not in trajectories_test.keys():
        trajectories_test[route] = np.zeros(72*7).reshape(7,72)
    trajectories_test[route][day][tw_n] = line['avg_travel_time']

In [106]:
x = trajectories_test['A2'][:,21]
p = test_data['A2'][:,21]
e = np.mean(np.abs((x-p)/x))
print e

1.0


In [107]:
tw = [18,19,20,21,22,23,45,46,47,48,49,50]
p = test_data
x = trajectories_test

sum = 0.0
n = 0.0

for route in trajectories_test.keys():
    for tw_n in tw:
        for day in range(7):
            if x[route][day][tw_n] != 0:
                xxx = x[route][day][tw_n]
                sum += np.abs((xxx - p[route][day][tw_n])/xxx)
                n += 1
e = sum / n
print e

1.0


In [108]:
sum = 0
for route in trajectories_train.keys():
    

IndentationError: expected an indented block (<ipython-input-108-40a0c35d7b83>, line 3)